# 1 start pods on minikube

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## add auth tokens for the controller of uoa-poc2

### create new `secrets/auth-tokens.json`

example)
```json
[
    {
        "host": ".*",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "1IqNHfjQsD84mPHvciATObXM3ozfHmX1",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }, {
                    "token": "mgMVtijGi6JWX9HT2PFXkZ6xqSdOZVVd",
                    "allowed_paths": ["^/controller/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }
]
```

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens|=.+[
  {
    "token": "'$(randomstr32)'",
    "allowed_paths": ["^/controller/.*$"]
  }
]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json

## change the auth-tokens to kubernetes secrets

### watch `auth` log

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "kubectl logs -f -lapp=auth --all-containers=true"

### delete and re-register auth-tokens to kubernetes secrets

In [ ]:
kubectl delete secret auth-tokens

In [ ]:
kubectl create secret generic auth-tokens --from-file=${CORE_ROOT}/secrets/auth-tokens.json

### confirm the token will be reloaded

**wait a few minutes until the change of secret is detected by Kubernetes.**  
When the new secret is detected, the tokens of auth will be reloaded automatically.

log messages like below will be shown after tokens is reloaded.

```
...
--------
2019/08/06 02:28:00 hosts: [.*]
--------
2019/08/06 02:28:00 bearerTokenAllowedPaths: map[.*:map[OnjhCAf8oIRpwBfbFDCh7hu5kh3e9TFM:[^/orion/.*$ ^/idas/.*$ ^/comet/.*$] BcC7SFz7tWAqNtsmQOqbfDCgXqAUyVyn:[^/controller/.*$]]]
--------
2019/08/06 02:28:00 basicAuthPaths, map[]
--------
2019/08/06 02:28:00 noAuthPaths, map[.*:[]]
--------
```

## start robot-controller on minikube

In [ ]:
kubectl apply -f controller/robot-controller-minikube-service.yaml

In [ ]:
envsubst < controller/robot-controller-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=robot-controller

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
robot-controller-5c7fd7df75-59f7s   1/1     Running   0          26s
robot-controller-5c7fd7df75-lwjs6   1/1     Running   0          26s
robot-controller-5c7fd7df75-wlxfq   1/1     Running   0          26s
```

In [ ]:
kubectl get services -l app=robot-controller

example)
```
NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
robot-controller   ClusterIP   10.101.151.122   <none>        3000/TCP   3m30s
```